In [ ]:
from player import Player

In [ ]:
import socceraction.spadl as spadl
import socceraction.xthreat as xthreat
import soccerdata as sd
from pathlib import PosixPath
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

import socceraction.spadl.config as spadlconfig
from socceraction.spadl.schema import SPADLSchema
from pandera.typing import DataFrame, Series
import numpy.typing as npt
from sklearn.exceptions import NotFittedError
from typing import Callable, List, Optional, Tuple
import os
from datetime import date, timedelta
from ipywidgets import interact

In [ ]:
def string_to_date(input):
    input_parts = input.split('-')
    return date(int(input_parts[0]), int(input_parts[1]), int(input_parts[2]))	

def get_time_list(start):
    date_list = []
    start_date = string_to_date(start)
    end_date = date.today()
    delta = timedelta(days=1)
    while start_date <= end_date:
        date_list.append(start_date.strftime("%Y-%m-%d"))
        start_date += delta
    return date_list

In [ ]:
def get_df():
    return pd.read_csv("../data/player_db.csv", sep=";")

def mov_avg(arr, win):
    avg_arr = []
    for x in range(len(arr)):
        if x == 0:
            continue
        if x < win:
            # print(x, " mean ", np.mean(arr[0:x]), " len ", len(arr[0:x]))
            avg_arr.append(np.mean(arr[0:x]))
        else:
            avg_arr.append(np.mean(arr[x-win:x]))
            # print(x, " mean ", np.mean(arr[x-win:x]), " len ", len(arr[x-win:x]))

    return avg_arr

def draw_plot(id):
    # id to id
    df = get_df()
    id = df[df["player_name"] == id]['id'].values[0]
    proto_player = Player().parse(open(f"./../data/{str(id)}.pb", "rb").read())
    df = pd.DataFrame(proto_player.starter.x_i)
    attack, defense, shots = [], [], []
    df.sort_values('game_date', inplace=True)
    date_list = get_time_list(df.loc[0]['game_date'])
    for x in range(len(df)):
        if x == 0:
            continue
        long_term = x - 50 if x - 50 >= 0 else 0
        medium_term = x - 25 if x - 25 >= 0 else 0
        short_term = x - 10 if x - 10 >= 0 else 0
        form = x - 3 if x - 3 >= 0 else 0
        attack.append(df.iloc[long_term:x]["x_t_pm"].mean() * 0.5 + 
                    df.iloc[medium_term:x]["x_t_pm"].mean() * 0.15 +
                    df.iloc[short_term:x]["x_t_pm"].mean() * 0.15 +
                    df.iloc[form:x]["x_t_pm"].mean() * 0.2)
    for x in range(len(df)):
        if x == 0:
            continue
        long_term = x - 50 if x - 50 >= 0 else 0
        medium_term = x - 25 if x - 25 >= 0 else 0
        short_term = x - 10 if x - 10 >= 0 else 0
        form = x - 3 if x - 3 >= 0 else 0
        defense.append(df.iloc[long_term:x]["x_d_pm"].mean() * 0.5 + 
                    df.iloc[medium_term:x]["x_d_pm"].mean() * 0.15 +
                    df.iloc[short_term:x]["x_d_pm"].mean() * 0.15 +
                    df.iloc[form:x]["x_d_pm"].mean() * 0.2)

    for x in range(len(df)):
        if x == 0:
            continue
        long_term = x - 50 if x - 50 >= 0 else 0
        medium_term = x - 25 if x - 25 >= 0 else 0
        short_term = x - 10 if x - 10 >= 0 else 0
        form = x - 3 if x - 3 >= 0 else 0
        shots.append(df.iloc[long_term:x]["x_g_pm"].mean() * 0.5 + 
                    df.iloc[medium_term:x]["x_g_pm"].mean() * 0.15 +
                    df.iloc[short_term:x]["x_g_pm"].mean() * 0.15 +
                    df.iloc[form:x]["x_g_pm"].mean() * 0.2)

    N = 10
    shots = mov_avg(shots, 10)
    attack = mov_avg(attack, 10)
    defense = mov_avg(defense, 10)
    plt.plot(np.arange(len(attack[0:10])), attack[0:10], '.g')
    plt.plot(np.arange(len(attack[0:10]), len(attack[0:10])+len(attack[10:])), attack[10:], 'g')
    plt.plot(np.arange(len(defense[0:10])), defense[0:10], '.r')
    plt.plot(np.arange(len(defense[0:10]), len(defense[0:10])+len(defense[10:])), defense[10:], 'r')
    plt.plot(np.arange(len(shots[0:10])), shots[0:10], '.b')
    plt.plot(np.arange(len(shots[0:10]), len(shots[0:10])+len(shots[10:])), shots[10:], 'b')
    # plt.plot(np.arange(len(attack)), np.convolve(attack, np.ones(N)/N, mode='same'), 'g')
    # plt.plot(np.arange(len(defense)), np.convolve(defense, np.ones(N)/N, mode='same'), 'r')
    # plt.plot(np.arange(len(shots)), np.convolve(shots, np.ones(N)/N, mode='same'), 'b')

df = get_df()
interact(draw_plot, id=df['player_name'].sort_values())

In [ ]:
# top players xT
names, value = [], []
for file in os.listdir("../data/"):
    if file == "player_db.csv":
        continue
    proto_player = Player().parse(open(f"./../data/{file}", "rb").read())
    df = pd.DataFrame(proto_player.starter.x_i)
    if df.shape[0] < 50:
        continue
    df.sort_values('game_date', inplace=True)
    
    val = ((df.iloc[-50:]["x_t_pm"] * df.iloc[-50:]["league_elo"]).mean() * 0.5 + 
           (df.iloc[-25:]["x_t_pm"] * df.iloc[-25:]["league_elo"]).mean() * 0.15 +
           (df.iloc[-10:]["x_t_pm"] * df.iloc[-10:]["league_elo"]).mean() * 0.15 +
           (df.iloc[-3:]["x_t_pm"] * df.iloc[-3:]["league_elo"]).mean() * 0.2)
    pid = proto_player.player_id
    name_db = pd.read_csv("../data/player_db.csv", sep=";")
    names.append(name_db[name_db['id'] == pid]['player_name'].values[0])
    value.append(val)

top_xT = pd.DataFrame({"name": names, "impact": value})

In [ ]:
top_xT.sort_values('impact', ascending=False, inplace=True)
top_xT.head(20)

In [ ]:
# top players shooting
names, value = [], []
for file in os.listdir("../data/"):
    if file == "player_db.csv":
        continue
    proto_player = Player().parse(open(f"./../data/{file}", "rb").read())
    df = pd.DataFrame(proto_player.starter.x_i)
    if df.shape[0] < 50:
        continue
    df.sort_values('game_date', inplace=True)
    
    val = ((df.iloc[-50:]["x_g_pm"] * df.iloc[-50:]["league_elo"]).mean() * 0.5 + 
           (df.iloc[-25:]["x_g_pm"] * df.iloc[-25:]["league_elo"]).mean() * 0.15 +
           (df.iloc[-10:]["x_g_pm"] * df.iloc[-10:]["league_elo"]).mean() * 0.15 +
           (df.iloc[-3:]["x_g_pm"] * df.iloc[-3:]["league_elo"]).mean() * 0.2)
    pid = proto_player.player_id
    name_db = pd.read_csv("../data/player_db.csv", sep=";")
    names.append(name_db[name_db['id'] == pid]['player_name'].values[0])
    value.append(val)

top_xG = pd.DataFrame({"name": names, "impact": value})

In [ ]:
top_xG.sort_values('impact', ascending=False, inplace=True)
top_xG.head(20)

In [ ]:
# top players shooting
names, value = [], []
for file in os.listdir("../data/"):
    if file == "player_db.csv":
        continue
    proto_player = Player().parse(open(f"./../data/{file}", "rb").read())
    df = pd.DataFrame(proto_player.starter.x_i)
    if df.shape[0] < 50:
        continue
    df.sort_values('game_date', inplace=True)
    
    val = ((df.iloc[-50:]["x_d_pm"] * df.iloc[-50:]["league_elo"]).mean() * 0.5 + 
           (df.iloc[-25:]["x_d_pm"] * df.iloc[-25:]["league_elo"]).mean() * 0.15 +
           (df.iloc[-10:]["x_d_pm"] * df.iloc[-10:]["league_elo"]).mean() * 0.15 +
           (df.iloc[-3:]["x_d_pm"] * df.iloc[-3:]["league_elo"]).mean() * 0.2)
    pid = proto_player.player_id
    name_db = pd.read_csv("../data/player_db.csv", sep=";")
    names.append(name_db[name_db['id'] == pid]['player_name'].values[0])
    value.append(val)

top_xD = pd.DataFrame({"name": names, "impact": value})

In [ ]:
top_xD.sort_values('impact', ascending=False, inplace=True)
top_xD.head(20)

In [ ]:
# top players shooting
names, value = [], []
for file in os.listdir("../data/"):
    if file == "player_db.csv":
        continue
    proto_player = Player().parse(open(f"./../data/{file}", "rb").read())
    df = pd.DataFrame(proto_player.starter.x_i)
    if df.shape[0] < 50:
        continue
    df.sort_values('game_date', inplace=True)
    
    val = ((df.iloc[-50:]["x_k_pm"] * df.iloc[-50:]["league_elo"]).mean() * 0.5 + 
           (df.iloc[-25:]["x_k_pm"] * df.iloc[-25:]["league_elo"]).mean() * 0.15 +
           (df.iloc[-10:]["x_k_pm"] * df.iloc[-10:]["league_elo"]).mean() * 0.15 +
           (df.iloc[-3:]["x_k_pm"] * df.iloc[-3:]["league_elo"]).mean() * 0.2)
    pid = proto_player.player_id
    name_db = pd.read_csv("../data/player_db.csv", sep=";")
    names.append(name_db[name_db['id'] == pid]['player_name'].values[0])
    value.append(val)

top_xK = pd.DataFrame({"name": names, "impact": value})

In [ ]:
top_xK.sort_values('impact', ascending=False, inplace=True)
top_xK.head(20)

In [ ]:
df = get_df()
id = 73084
proto_player = Player().parse(open(f"./../data/{str(id)}.pb", "rb").read())
df = pd.DataFrame(proto_player.starter.x_i)
plt.scatter(np.arange(df["x_t_pm"].shape[0]),df["x_t_pm"])
from scipy import stats
z = np.abs(stats.zscore(df["x_t_pm"]))

In [ ]:
df_o = df[(z < 1.5)]
df_m = df[(z >= 1.5)]
df_o.shape
df_m.head()

plt.scatter(df_o.index, df_o['x_t_pm'])
plt.scatter(df_m.index, df_m['x_t_pm'], color="r")
m, b = np.polyfit(df_o.index.values, df_o['x_t_pm'], 1)
plt.plot(df_o.index.values, m*df_o.index.values+b)

In [ ]:
df = get_df()
id = 11119
proto_player = Player().parse(open(f"./../data/{str(id)}.pb", "rb").read())
df = pd.DataFrame(proto_player.starter.x_i)
plt.scatter(df.index, df["x_g_pm"])
from scipy import stats
z = np.abs(stats.zscore(df["x_g_pm"]))
m, b = np.polyfit(df.index.values[-10:], df['x_g_pm'].iloc[-10:], 1)
plt.plot(df.index.values[-10:], m*df.index.values[-10:]+b)

In [ ]:
df_o = df[(z < 1.5)]
df_m = df[(z >= 1.5)]
df_o.shape
df_m.head()

plt.scatter(df_o.index, df_o['x_g_pm'])
plt.scatter(df_m.index, df_m['x_g_pm'], color="r")
m, b = np.polyfit(df_o.index.values, df_o['x_g_pm'], 1)
plt.plot(df_o.index.values, m*df_o.index.values+b)

In [ ]:
df

In [ ]:
plt.scatter(np.arange(df["g_i"].shape[0]), df["g_i"])
m, b = np.polyfit(df.index.values, df['g_i'], 1)
plt.plot(df.index.values, m*df['g_i'].index.values+b)

In [ ]:
df['g_i'].mean()

In [ ]:
m*(df['g_i'].index.values[-1])+b